In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import iit
print(iit.__file__)

/Users/evananders/far_cluster/iit/iit/__init__.py


In [3]:
import transformer_lens as tl
import numpy as np
import torch as t
import wandb

from iit.utils.iit_dataset import train_test_split
from iit.utils.iit_dataset import IITDataset
from iit.utils.correspondence import Correspondence
from iit.utils.argparsing import IOIArgParseNamespace

device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [4]:
D_MODEL = 48
N_CTX = 23
N_LAYERS = 3
N_HEADS = 4
D_VOCAB = 4

#Want to specify this somewhere central, e.g., iit.tasks.ioi but iit.tasks.parens
ll_cfg = tl.HookedTransformerConfig(
        n_layers = N_LAYERS,
        d_model = D_MODEL,
        n_ctx = N_CTX,
        d_head = D_MODEL // N_HEADS,
        d_vocab = D_VOCAB,
        act_fn = "relu",
)

ll_model = tl.HookedTransformer(ll_cfg).to(device)

Moving model to device:  cpu


In [5]:
from paren_checker import HighLevelParensBalanceChecker, TwoTaskParensDataset, LeftGreaterParensDataset, BalancedParensDataset
from torch.utils.data import Dataset

hl_model = HighLevelParensBalanceChecker(device=device)
dataset = TwoTaskParensDataset(
    N_samples = 20_000,
    n_ctx = N_CTX,
    seed = 42,
)

class CustomDataset(Dataset):
    def __init__(self, data, targets, markers):
        """
        Args:
            data (list or numpy array): List or array of input data.
            targets (list or numpy array): List or array of target data.
        """
        self.data = t.tensor(data).to(int)
        self.targets = t.tensor(targets).to(int)
        self.markers = t.tensor(markers).to(int)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index
        Returns:
            tuple: (input tensor, target tensor)
        """
        return self.data[idx], self.targets[idx], self.markers[idx]


decorated_dset = CustomDataset(
    data = dataset.get_dataset()['tokens'],
    targets = np.array(dataset.get_dataset()['labels'])[:, None],
    markers = np.array(dataset.get_dataset()['markers'])[:, None]
)


print("making IIT dataset")
train_dataset, test_dataset = train_test_split(
    decorated_dset, test_size=0.2, random_state=42
)
train_set = IITDataset(train_dataset, train_dataset, seed=0)
test_set = IITDataset(test_dataset, test_dataset, seed=0)

making IIT dataset


In [6]:
print(train_set[0])

((tensor([3, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2]), tensor([0]), tensor([0])), (tensor([3, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2]), tensor([0]), tensor([2])))


In [39]:
from typing import Callable
from dataclasses import asdict
from iit.utils.index import Ix, TorchIndex
from iit_repo_paren_checker import ParensModelPair

all_attns = [f"blocks.{i}.attn.hook_z" for i in range(ll_cfg.n_layers)]
all_mlps = [f"blocks.{i}.mlp.hook_post" for i in range(ll_cfg.n_layers)]
all_nodes_hook = "blocks.0.hook_resid_pre"
head_index: Callable[[int], TorchIndex] = lambda idx: Ix[[None, None, idx, None]]
#make correlation
corr_dict = {
        'input_hook' :           [(all_nodes_hook,  Ix[[None]], None)],
        'left_parens_hook' :     [(all_attns[0],    head_index(0), None)],
        'right_parens_hook' :    [(all_attns[0],    head_index(1), None)],
        'task_hook':             [(all_attns[0],    head_index(2), None)],
        'mlp0_hook':             [(all_mlps[0],     Ix[[None]], None)],
        'mlp1_hook' :            [(all_mlps[1],     Ix[[None]], None)],
        'horizon_lookback_hook': [(all_attns[2],    head_index(3), None)],
        'output_check_hook' :    [(all_mlps[2],     Ix[[None]], None)]
    }


print("making model pair")
corr = Correspondence.make_corr_from_dict(corr_dict, suffixes={"attn": "attn.hook_z", "mlp": "mlp.hook_post"})

train_args = IOIArgParseNamespace(
    include_mlp = True,
    use_wandb = False,
    num_samples = 20_000,
    batch_size = 128,
    next_token = False,
    
    epochs = 100,
    lr = 3e-5,
    iit = 1,
    b = 0.5,
    s = 0.5,
    clip_grad_norm = False,
    use_single_loss = True,
    save_to_wandb = False,
    device = device,
)
train_args_dict = asdict(train_args)
train_args_dict['siit_sample_strategy'] = 'sample_all' #or individual

model_pair = ParensModelPair(
    ll_model=ll_model,
    hl_model=hl_model,
    corr=corr,
    training_args=train_args_dict,
)
# for k, v in model_pair.training_args.items():
#     print(k, v)

making model pair
{'input_hook': HookPoint(), 'left_parens_hook': HookPoint(), 'right_parens_hook': HookPoint(), 'task_hook': HookPoint(), 'greater_hook': HookPoint(), 'elevation_hook': HookPoint(), 'mlp0_hook': HookPoint(), 'mlp1_hook': HookPoint(), 'horizon_lookback_hook': HookPoint(), 'output_check_hook': HookPoint()}
dict_keys([input_hook, left_parens_hook, right_parens_hook, task_hook, mlp0_hook, mlp1_hook, horizon_lookback_hook, output_check_hook])


In [40]:
for node in model_pair.nodes_not_in_circuit:
    print(node)

LLNode(name='blocks.0.attn.hook_z', index=[:, :, 3, :], subspace=None)
LLNode(name='blocks.1.attn.hook_z', index=[:, :, 0, :], subspace=None)
LLNode(name='blocks.1.attn.hook_z', index=[:, :, 1, :], subspace=None)
LLNode(name='blocks.1.attn.hook_z', index=[:, :, 2, :], subspace=None)
LLNode(name='blocks.1.attn.hook_z', index=[:, :, 3, :], subspace=None)
LLNode(name='blocks.2.attn.hook_z', index=[:, :, 0, :], subspace=None)
LLNode(name='blocks.2.attn.hook_z', index=[:, :, 1, :], subspace=None)
LLNode(name='blocks.2.attn.hook_z', index=[:, :, 2, :], subspace=None)


In [41]:
print("training model pair")
model_pair.train(train_set, test_set, epochs=train_args.epochs, use_wandb=train_args.use_wandb, optimizer_cls=t.optim.AdamW, optimizer_kwargs={'weight_decay' : 1e-3})

training model pair
training_args={'next_token': False, 'non_ioi_thresh': 0.65, 'use_per_token_check': False, 'batch_size': 128, 'lr': 3e-05, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': None, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'clip_grad_norm': False, 'seed': 0, 'detach_while_caching': True, 'atol': 0.05, 'use_single_loss': True, 'iit_weight': 1.0, 'behavior_weight': 1.0, 'strict_weight': 1.0, 'output_dir': './results', 'include_mlp': True, 'use_wandb': False, 'num_samples': 20000, 'device': device(type='cpu'), 'weights': '100_100_40', 'mean': True, 'load_from_wandb': False, 'epochs': 100, 'iit': 1, 'b': 0.5, 's': 0.5, 'save_to_wandb': False, 'siit_sample_strategy': 'sample_all'}


  1%|          | 1/100 [00:30<50:26, 30.57s/it]


Epoch 0: train/iit_loss: 0.0314, train/behavior_loss: 0.0014, train/strict_loss: 0.0026, val/iit_loss: 0.0343, val/IIA: 98.75%, val/accuracy: 100.00%, val/strict_accuracy: 99.99%, 


  2%|▏         | 2/100 [01:00<49:03, 30.04s/it]


Epoch 1: train/iit_loss: 0.0280, train/behavior_loss: 0.0014, train/strict_loss: 0.0031, val/iit_loss: 0.0203, val/IIA: 99.34%, val/accuracy: 100.00%, val/strict_accuracy: 99.99%, 


...


Epoch 97: train/iit_loss: 0.0089, train/behavior_loss: 0.0001, train/strict_loss: 0.0004, val/iit_loss: 0.0071, val/IIA: 99.73%, val/accuracy: 100.00%, val/strict_accuracy: 100.00%, 


 99%|█████████▉| 99/100 [44:13<00:26, 26.21s/it]


Epoch 98: train/iit_loss: 0.0061, train/behavior_loss: 0.0001, train/strict_loss: 0.0001, val/iit_loss: 0.0050, val/IIA: 99.85%, val/accuracy: 100.00%, val/strict_accuracy: 100.00%, 


100%|██████████| 100/100 [44:39<00:00, 26.79s/it]


Epoch 99: train/iit_loss: 0.0059, train/behavior_loss: 0.0001, train/strict_loss: 0.0001, val/iit_loss: 0.0073, val/IIA: 99.76%, val/accuracy: 100.00%, val/strict_accuracy: 100.00%, 
